In [36]:
%load_ext sql
%sql sqlite:///dataset_1.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Activity 1: Creating tables in SQL
------------

The database loaded has one table, `precipitation_full`, having the following schema:

> * `state_code`
> * `station_id`
> * `year`
> * `month`
> * `day`
> * `hour`
> * `precipitation`
> * `flag_1`
> * `flag_2`

Each tuple in this table describes one hour of rainfall (`precipitation`- in hundredths of an inch) at one station (`station_id`) in one state (`state_code`).  Note that tuples with `hour=25` record the total rainfall for that day, and that we can ignore the values of attributes `flag_1` and `flag_2` for now.

**1-1 View the schema of existing tables:**

In [37]:
%sql PRAGMA table_info(precipitation_full);

 * sqlite:///dataset_1.db
Done.


cid,name,type,notnull,dflt_value,pk
0,state_code,INT,0,None,0
1,station_id,INT,0,None,0
2,year,INT,0,None,0
3,month,INT,0,None,0
4,day,INT,0,None,0
5,hour,INT,0,None,0
6,precipitation,INT,0,None,0
7,flag_1,VARCHAR(1),0,None,0
8,flag_2,VARCHAR(1),0,None,0


**1-2 Create tables:**

In [38]:
%%sql DROP TABLE IF EXISTS MS205;
CREATE TABLE MS205
(
    student_id  INT           PRIMARY KEY,
    station_id  INT           NOT NULL,
    start_hour  INT           NOT NULL,
    end_hour    INT           NOT NULL,
    holding_cup BOOLEAN       NOT NULL,
    off_day     INT           NOT NULL,
    cup_size        FLOAT         NOT NULL,
    dept        INT           NOT NULL,
    name        VARCHAR(1000)
);

 * sqlite:///dataset_1.db
Done.
Done.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

**1-3 View the schema of created tables:**

In [39]:
%sql SELECT sql FROM sqlite_master WHERE name = 'MS205';

 * sqlite:///dataset_1.db
Done.


sql
"CREATE TABLE MS205( student_id INT PRIMARY KEY, station_id INT NOT NULL, start_hour INT NOT NULL, end_hour INT NOT NULL, holding_cup BOOLEAN NOT NULL, off_day INT NOT NULL, cup_size FLOAT NOT NULL, dept INT NOT NULL, name VARCHAR(1000))"


In [40]:
%sql PRAGMA table_info(MS205);

 * sqlite:///dataset_1.db
Done.


cid,name,type,notnull,dflt_value,pk
0,student_id,INT,0,None,1
1,station_id,INT,1,None,0
2,start_hour,INT,1,None,0
3,end_hour,INT,1,None,0
4,holding_cup,BOOLEAN,1,None,0
5,off_day,INT,1,None,0
6,cup_size,FLOAT,1,None,0
7,dept,INT,1,None,0
8,name,VARCHAR(1000),0,None,0


Activity 2: Single table queries
-------------

In [41]:
%%sql
drop table if exists product; -- This needs to be dropped if exists, see why further down!
drop table if exists company;
pragma foreign_keys = ON; -- WARNING by default off in sqlite

create table company (
    cname varchar primary key, -- company name uniquely identifies the company.
    stockprice money, -- stock price is in money 
    country varchar); -- country is just a string
insert into company values ('ToyWorks', 25.0, 'USA');
insert into company values ('ToyFriends', 65.0, 'China');
insert into company values ('ToyCo', 15.0, 'China');

create table product(
       pname varchar, -- name of the product
       price money, -- price of the product
       category varchar, -- category
       manufacturer varchar, -- manufacturer
       primary key (pname, manufacturer),
       foreign key (manufacturer) references company(cname));
insert into product values('Pikachu', 19.99, 'Toy', 'ToyWorks');
insert into product values('Pikachu', 19.99, 'Toy', 'ToyFriends');
insert into product values('Pokeball', 29.99, 'Electronic', 'ToyCo');
insert into product values('Bulbasaur', 149.99, 'Toy', 'ToyFriends');
insert into product values('Charizard', 203.99, 'Toy', 'ToyCo');
insert into product values('PokeCamera', 19.99, 'Electronic', 'ToyWorks');

 * sqlite:///dataset_1.db
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [42]:
%sql select * from product;

 * sqlite:///dataset_1.db
Done.


pname,price,category,manufacturer
Pikachu,19.99,Toy,ToyWorks
Pikachu,19.99,Toy,ToyFriends
Pokeball,29.99,Electronic,ToyCo
Bulbasaur,149.99,Toy,ToyFriends
Charizard,203.99,Toy,ToyCo
PokeCamera,19.99,Electronic,ToyWorks


In [43]:
%sql Select distinct pname, price from product where pname LIKE '%pika%';

 * sqlite:///dataset_1.db
Done.


pname,price
Pikachu,19.99


Activity 3: Multi-table queries
-------------

In [44]:
%%sql
SELECT DISTINCT P.category
FROM Product AS P, Company AS C
WHERE country='China' AND P.manufacturer=C.cname

 * sqlite:///dataset_1.db
Done.


category
Toy
Electronic


Activity 4: Aggregation operators, GROUP BY
------------

A set of tables that describe the up-and-coming bagel startup industry;

`bagel`, which describes types of bagels made by the different bagel companies:
> * name STRING
> * price FLOAT
> * made_by STRING

`purchase`:
> * bagel_name STRING
> * franchise STRING
> * date INT
> * quantity INT
> * purchaser_age INT

Where `purchase.bagel_name` references `bagel.name` and `purchase.franchise` references `bagel.made_by`:

In [45]:
%sql SELECT * FROM bagel;

 * sqlite:///dataset_1.db
Done.


name,price,made_by
Plain with shmear,1.99,Bobs Bagels
Egg with shmear,2.39,Bobs Bagels
eBagel Drinkable Bagel,27.99,eBagel
eBagel Expansion Pack,1.99,eBagel
Plain with shmear,0.99,BAGEL CORP
Organic Flax-seed bagel chips,0.99,BAGEL CORP


In [46]:
%sql SELECT * FROM purchase;

 * sqlite:///dataset_1.db
Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17
eBagel Expansion Pack,eBagel,1,137,5
Plain with shmear,Bobs Bagels,4,24,None
Moonshine,BAGEL CORP,7,1000,37


**4-1 The total revenue for each bagel type which had an average purchaser age over 18:**

In [47]:
%%sql
select bagel_name, round(sum(quantity*price), 2) as total_revenue
from bagel as b, purchase as p
where bagel_name=name and franchise=made_by
group by (bagel_name)
having avg(purchaser_age) > 18;

 * sqlite:///dataset_1.db
Done.


bagel_name,total_revenue
Egg with shmear,14.34
Plain with shmear,84.51


**4-2 Get station_ids which have average precipitations > 75:**

In [48]:
%sql SELECT * FROM precipitation_full LIMIT 5;

 * sqlite:///dataset_1.db
Done.


state_code,station_id,year,month,day,hour,precipitation,flag_1,flag_2
1,807,2013,9,1,1,None,[,
1,807,2013,9,1,25,0,I,
1,807,2013,9,30,24,None,],
1,807,2013,9,30,25,0,I,
1,6303,2013,9,1,1,0,g,


**4-2-1 Nested way:**

In [49]:
%time %sql select DISTINCT p.station_id from precipitation p where (select avg(precipitation) from precipitation where station_id = p.station_id) > 75;

 * sqlite:///dataset_1.db
Done.
Wall time: 17 ms


station_id
88302
250002
335701
357302
488301


**4-2-2 Group-by way:**

In [50]:
%time %sql select station_id from precipitation group by (station_id) having avg(precipitation) > 75;

 * sqlite:///dataset_1.db
Done.
Wall time: 1.99 ms


station_id
88302
250002
335701
357302
488301


Activity 5: Quantifiers, NULLs, and Outer Joins
------------
Using a single SQL query to find out if there were any purchases of products not on one of the company's official lists (i.e. the bagel table):

In [51]:
%sql select p.bagel_name, b.name, p.franchise from purchase p left outer join bagel b on (franchise=made_by and p.bagel_name=b.name) where p.bagel_name not in(select name from bagel);

 * sqlite:///dataset_1.db
Done.


bagel_name,name,franchise
Moonshine,None,BAGEL CORP
